# Shuttlt bus data analysis

### note: not calculate distance and predict by original data 

In [1]:
%matplotlib inline

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math as math
import plotly
from datetime import tzinfo, timedelta, datetime

### load file

In [3]:
#get the stop segment
csv_stop = pd.read_csv("./../data/iot.stops.csv",sep = '\t',header = 0)
csv_route = pd.read_csv("./../data/iot.routes.csv",sep = '\t',header = 0)

#for calculate the distances between 2 stops 
csv_gps = pd.read_csv("./../data/iot.locations.csv",sep = '\t',header = 0)

In [4]:
print(csv_stop.shape)
csv_stop.head()

(167, 6)


,_id,isGoToWork,latitude,longitude,stopName,updated
0,5ae83320748fb2a67c5f6e54,True,22.249541,113.520559,明珠中,2018-05-01T09:28:00.844Z
1,5ae8334c748fb2a67c5f6e55,True,22.256653,113.519771,翠微,2018-05-01T09:28:44.366Z
2,5ae8336c748fb2a67c5f6e56,True,22.267762,113.536134,仁恒星园,2018-05-01T09:29:16.147Z
3,5ae834df567e17937cf407e1,True,22.268722,113.542062,香洲区府,2018-05-01T09:35:27.388Z
4,5ae83521567e17937cf407e2,True,22.269261,113.545517,南村,2018-05-01T09:36:33.812Z


In [5]:
print(csv_route.shape)
csv_route.head()

(199, 8)


,_id,busId,isGo2Work,plannedArrivalTime,routeNumber,seqNumber,stopId,updated
0,5ae83df03023169da447c2b6,5ae8393c3023169da447c2ae,True,7:38,1,5,5ae83521567e17937cf407e2,2018-05-01T10:14:08.961Z
1,5ae83e6c3023169da447c2b9,5ae8399c3023169da447c2af,True,7:33,2,2,5ae83590567e17937cf407e5,2018-05-01T10:16:12.721Z
2,5ae83bdf3023169da447c2b1,5ae8393c3023169da447c2ae,True,7:23,1,1,5ae83320748fb2a67c5f6e54,2018-05-01T10:05:19.471Z
3,5ae83e563023169da447c2b8,5ae8399c3023169da447c2af,True,7:31,2,1,5ae8355a567e17937cf407e4,2018-05-01T10:15:50.716Z
4,5ae83e823023169da447c2ba,5ae8399c3023169da447c2af,True,7:34,2,3,5ae835ad567e17937cf407e6,2018-05-01T10:16:34.087Z


In [6]:
print(csv_gps.shape)
csv_gps.head()

(46239, 5)


,bus,device,latitude,longitude,updated
0,1,stm32,22.251497,113.517815,2018-05-20T10:43:01.277Z
1,1,stm32,22.251497,113.517807,2018-05-20T10:43:17.674Z
2,1,stm32,22.251528,113.517891,2018-05-20T10:43:33.736Z
3,1,stm32,22.251572,113.517883,2018-05-20T10:43:49.878Z
4,1,stm32,22.251566,113.517883,2018-05-20T10:44:06.055Z


## Merge route and stop according to the stopId

In [7]:
route_stop_set = csv_route.merge(csv_stop, left_on='stopId', right_on='_id', how='left')
route_stop_set = route_stop_set[['isGo2Work', 'plannedArrivalTime', 'routeNumber', 'seqNumber','latitude','longitude','stopName']]
# route_stop_set.columns = []
route_stop_set = route_stop_set.sort_values(by=['routeNumber','isGo2Work','seqNumber']).reset_index()
print(route_stop_set.shape)
route_stop_set.head()

(199, 8)


,index,isGo2Work,plannedArrivalTime,routeNumber,seqNumber,latitude,longitude,stopName
0,98,False,18:00,1,1,22.372631,113.575924,南方软件园
1,99,False,18:00,1,2,22.269416,113.545678,南村
2,100,False,18:00,1,3,22.268548,113.540206,香洲区府
3,101,False,18:00,1,4,22.267938,113.536273,仁恒星园
4,102,False,18:00,1,5,22.267083,113.530912,安居园


In [8]:
route_stop_set['next_seqNumber'] = route_stop_set.seqNumber + 1

In [9]:
route_stop_pair_set = route_stop_set.merge(route_stop_set, left_on=['routeNumber','isGo2Work','next_seqNumber'],\
                                           right_on=['routeNumber','isGo2Work','seqNumber'], how='left')
route_stop_pair_set = route_stop_pair_set[['routeNumber','isGo2Work','stopName_x','stopName_y','seqNumber_x','next_seqNumber_x',\
                                          'latitude_x','longitude_x','latitude_y','longitude_y','plannedArrivalTime_y']]
route_stop_pair_set.columns = ['routeNumber','isGo2Work','stopName','next_stopName','seqNumber','next_seqNumber','latitude','longitude',\
                               'next_latitude','next_longitude','plannedArrivalTime']

In [10]:
print(route_stop_pair_set.shape)
route_stop_pair_set.head(10)

(199, 11)


,routeNumber,isGo2Work,stopName,next_stopName,seqNumber,next_seqNumber,latitude,longitude,next_latitude,next_longitude,plannedArrivalTime
0,1,False,南方软件园,南村,1,2,22.372631,113.575924,22.269416,113.545678,18:00
1,1,False,南村,香洲区府,2,3,22.269416,113.545678,22.268548,113.540206,18:00
2,1,False,香洲区府,仁恒星园,3,4,22.268548,113.540206,22.267938,113.536273,18:00
3,1,False,仁恒星园,安居园,4,5,22.267938,113.536273,22.267083,113.530912,18:00
4,1,False,安居园,翠微,5,6,22.267083,113.530912,22.257149,113.519530,18:00
5,1,False,翠微,明珠中,6,7,22.257149,113.519530,22.249434,113.520293,18:00
6,1,False,明珠中,NaN,7,8,22.249434,113.520293,NaN,NaN,NaN
7,1,True,明珠中,翠微,1,2,22.249541,113.520559,22.256653,113.519771,7:26
8,1,True,翠微,仁恒星园,2,3,22.256653,113.519771,22.267762,113.536134,7:34
9,1,True,仁恒星园,香洲区府,3,4,22.267762,113.536134,22.268722,113.542062,7:37


In [39]:
route_stop_pair_set = route_stop_pair_set.dropna()
print(route_stop_pair_set.shape)
route_stop_pair_set.head()

(171, 11)


,routeNumber,isGo2Work,stopName,next_stopName,seqNumber,next_seqNumber,latitude,longitude,next_latitude,next_longitude,plannedArrivalTime
0,1,False,南方软件园,南村,1,2,22.372631,113.575924,22.269416,113.545678,18:00
1,1,False,南村,香洲区府,2,3,22.269416,113.545678,22.268548,113.540206,18:00
2,1,False,香洲区府,仁恒星园,3,4,22.268548,113.540206,22.267938,113.536273,18:00
3,1,False,仁恒星园,安居园,4,5,22.267938,113.536273,22.267083,113.530912,18:00
4,1,False,安居园,翠微,5,6,22.267083,113.530912,22.257149,113.519530,18:00


In [11]:
csv_gps.updated = pd.to_datetime(csv_gps.updated,utc = True)
csv_gps.head()

,bus,device,latitude,longitude,updated
0,1,stm32,22.251497,113.517815,2018-05-20 10:43:01.277
1,1,stm32,22.251497,113.517807,2018-05-20 10:43:17.674
2,1,stm32,22.251528,113.517891,2018-05-20 10:43:33.736
3,1,stm32,22.251572,113.517883,2018-05-20 10:43:49.878
4,1,stm32,22.251566,113.517883,2018-05-20 10:44:06.055


In [12]:
# csv_gps['record_time'] = csv_gps.updated.apply(lambda x: x.time())
csv_gps['record_time'] = csv_gps.updated.dt.time
csv_gps = csv_gps[csv_gps.device != 'arduino' ]
csv_gps.head(20)

,bus,device,latitude,longitude,updated,record_time
0,1,stm32,22.251497,113.517815,2018-05-20 10:43:01.277,10:43:01.277000
1,1,stm32,22.251497,113.517807,2018-05-20 10:43:17.674,10:43:17.674000
2,1,stm32,22.251528,113.517891,2018-05-20 10:43:33.736,10:43:33.736000
3,1,stm32,22.251572,113.517883,2018-05-20 10:43:49.878,10:43:49.878000
4,1,stm32,22.251566,113.517883,2018-05-20 10:44:06.055,10:44:06.055000
474,1,stm32,22.251497,113.516144,2018-05-21 23:21:32.022,23:21:32.022000
475,1,stm32,22.251520,113.515953,2018-05-21 23:21:46.123,23:21:46.123000
476,1,stm32,22.251549,113.515785,2018-05-21 23:22:00.263,23:22:00.263000
477,1,stm32,22.251701,113.515640,2018-05-21 23:22:14.362,23:22:14.362000
478,1,stm32,22.251860,113.515617,2018-05-21 23:22:30.501,23:22:30.501000


In [13]:
go_to_work_gps_set = csv_gps[((csv_gps.record_time > pd.to_datetime('23:00',utc = True).time())&(csv_gps.record_time < pd.to_datetime('23:59:59.9999',utc = True).time()))\
|((csv_gps.record_time > pd.to_datetime('00:00',utc = True).time())&(csv_gps.record_time < pd.to_datetime('00:40',utc = True).time()))] 
go_to_work_gps_set.insert(len(go_to_work_gps_set.columns),'isGo2work',True)
print(go_to_work_gps_set.shape)
go_to_work_gps_set.head()

(10919, 7)


,bus,device,latitude,longitude,updated,record_time,isGo2work
474,1,stm32,22.251497,113.516144,2018-05-21 23:21:32.022,23:21:32.022000,True
475,1,stm32,22.251520,113.515953,2018-05-21 23:21:46.123,23:21:46.123000,True
476,1,stm32,22.251549,113.515785,2018-05-21 23:22:00.263,23:22:00.263000,True
477,1,stm32,22.251701,113.515640,2018-05-21 23:22:14.362,23:22:14.362000,True
478,1,stm32,22.251860,113.515617,2018-05-21 23:22:30.501,23:22:30.501000,True


In [14]:
back_to_home_gps_set = csv_gps[(csv_gps.record_time > pd.to_datetime('9:50',utc = True).time())\
                               &(csv_gps.record_time < pd.to_datetime('12:00',utc = True).time())]
back_to_home_gps_set.insert(len(back_to_home_gps_set.columns),'isGo2work',False)
print(back_to_home_gps_set.shape)
back_to_home_gps_set.head()

(2336, 7)


,bus,device,latitude,longitude,updated,record_time,isGo2work
726,1,stm32,22.270960,113.531372,2018-05-22 10:50:00.439,10:50:00.439000,False
727,1,stm32,22.270912,113.530952,2018-05-22 10:50:17.512,10:50:17.512000,False
728,1,stm32,22.270786,113.530106,2018-05-22 10:50:33.614,10:50:33.614000,False
729,1,stm32,22.270592,113.528954,2018-05-22 10:50:49.734,10:50:49.734000,False
730,1,stm32,22.270554,113.528717,2018-05-22 10:51:06.760,10:51:06.760000,False


In [15]:
full_gps_set = go_to_work_gps_set
# full_gps_set = go_to_work_gps_set.append(back_to_home_gps_set)
print(full_gps_set.shape)
full_gps_set.head()

(10919, 7)


,bus,device,latitude,longitude,updated,record_time,isGo2work
474,1,stm32,22.251497,113.516144,2018-05-21 23:21:32.022,23:21:32.022000,True
475,1,stm32,22.251520,113.515953,2018-05-21 23:21:46.123,23:21:46.123000,True
476,1,stm32,22.251549,113.515785,2018-05-21 23:22:00.263,23:22:00.263000,True
477,1,stm32,22.251701,113.515640,2018-05-21 23:22:14.362,23:22:14.362000,True
478,1,stm32,22.251860,113.515617,2018-05-21 23:22:30.501,23:22:30.501000,True


In [44]:
def mergeRouteDataIntoGPSRecord(gps_record):
    reoute_stop_recod = route_stop_pair_set[(route_stop_pair_set.isGo2Work==gps_record.isGo2work) & (route_stop_pair_set.routeNumber == gps_record.bus) &\
                                           (gps_record.latitude >= route_stop_pair_set.latitude) & (gps_record.latitude <= route_stop_pair_set.next_latitude)]
    
    if(reoute_stop_recod.shape[0] == 0):
        reoute_stop_recod.loc[0] = [ np.nan for n in range(len(route_stop_pair_set.columns))]
    
    new_gps_record = pd.concat([gps_record,reoute_stop_recod.iloc[0]])
        
    return new_gps_record

In [45]:
start = datetime.now()
full_data_set = full_gps_set.apply(lambda x: mergeRouteDataIntoGPSRecord(x), axis=1)
print('Duration:',(datetime.now() - start).total_seconds())
full_data_set.head()

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy



Duration: 82.379237


,bus,device,latitude,longitude,updated,record_time,isGo2work,routeNumber,isGo2Work,stopName,next_stopName,seqNumber,next_seqNumber,latitude,longitude,next_latitude,next_longitude,plannedArrivalTime
474,1,stm32,22.251497,113.516144,2018-05-21 23:21:32.022,23:21:32.022000,True,1.0,True,明珠中,翠微,1.0,2.0,22.249541,113.520559,22.256653,113.519771,7:26
475,1,stm32,22.251520,113.515953,2018-05-21 23:21:46.123,23:21:46.123000,True,1.0,True,明珠中,翠微,1.0,2.0,22.249541,113.520559,22.256653,113.519771,7:26
476,1,stm32,22.251549,113.515785,2018-05-21 23:22:00.263,23:22:00.263000,True,1.0,True,明珠中,翠微,1.0,2.0,22.249541,113.520559,22.256653,113.519771,7:26
477,1,stm32,22.251701,113.515640,2018-05-21 23:22:14.362,23:22:14.362000,True,1.0,True,明珠中,翠微,1.0,2.0,22.249541,113.520559,22.256653,113.519771,7:26
478,1,stm32,22.251860,113.515617,2018-05-21 23:22:30.501,23:22:30.501000,True,1.0,True,明珠中,翠微,1.0,2.0,22.249541,113.520559,22.256653,113.519771,7:26


In [46]:
full_data_set = full_data_set.dropna()

In [47]:
full_data_set.shape

(9830, 18)

In [54]:
full_data_set[20:30]

,bus,device,latitude,longitude,updated,record_time,isGo2work,routeNumber,isGo2Work,stopName,next_stopName,seqNumber,next_seqNumber,latitude,longitude,next_latitude,next_longitude,plannedArrivalTime
494,1,stm32,22.259424,113.514679,2018-05-21 23:26:57.254,23:26:57.254000,True,1.0,True,翠微,仁恒星园,2.0,3.0,22.256653,113.519771,22.267762,113.536134,7:34
495,1,stm32,22.259394,113.514748,2018-05-21 23:27:13.316,23:27:13.316000,True,1.0,True,翠微,仁恒星园,2.0,3.0,22.256653,113.519771,22.267762,113.536134,7:34
496,1,stm32,22.259502,113.514832,2018-05-21 23:27:29.453,23:27:29.453000,True,1.0,True,翠微,仁恒星园,2.0,3.0,22.256653,113.519771,22.267762,113.536134,7:34
497,1,stm32,22.260126,113.514557,2018-05-21 23:27:45.489,23:27:45.489000,True,1.0,True,翠微,仁恒星园,2.0,3.0,22.256653,113.519771,22.267762,113.536134,7:34
498,1,stm32,22.261967,113.514381,2018-05-21 23:28:03.761,23:28:03.761000,True,1.0,True,翠微,仁恒星园,2.0,3.0,22.256653,113.519771,22.267762,113.536134,7:34
499,1,stm32,22.263474,113.514214,2018-05-21 23:28:21.161,23:28:21.161000,True,1.0,True,翠微,仁恒星园,2.0,3.0,22.256653,113.519771,22.267762,113.536134,7:34
500,1,stm32,22.269653,113.516968,2018-05-21 23:31:05.285,23:31:05.285000,True,1.0,True,南村,兴业中,5.0,6.0,22.269261,113.545517,22.276603,113.557696,7:40
501,1,stm32,22.269661,113.517128,2018-05-21 23:31:21.561,23:31:21.561000,True,1.0,True,南村,兴业中,5.0,6.0,22.269261,113.545517,22.276603,113.557696,7:40
502,1,stm32,22.268503,113.516731,2018-05-21 23:31:37.901,23:31:37.901000,True,1.0,True,仁恒星园,香洲区府,3.0,4.0,22.267762,113.536134,22.268722,113.542062,7:37
503,1,stm32,22.268526,113.516960,2018-05-21 23:31:53.912,23:31:53.912000,True,1.0,True,仁恒星园,香洲区府,3.0,4.0,22.267762,113.536134,22.268722,113.542062,7:37


## 1. data display

### @functions

In [20]:
# #坐标系转平面坐标
# def mct2x(lat):
#     x = lat/ 20037508.3427892 * 180 
#     return x
# def mct2y(lon):
#     result = []
#     for i in lon:
#         y = i/ 20037508.3427892 * 180 
#         y = 180 / math.pi * (2 * math.atan(math.exp(y * math.pi / 180)) - math.pi / 2) 
#         result.append(y)
#     return result

# def calculate_distance(lat1,lng1,lat2,lng2):
#     radlat1= math.radians(lat1)  
#     radlat2=math.radians(lat2)  
#     a=radlat1-radlat2  
#     b=math.radians(lng1)-math.radians(lng2)  
#     s=2*math.asin(math.sqrt(math.pow(math.sin(a/2),2)+math.cos(radlat1)*math.cos(radlat2)*math.pow(math.sin(b/2),2)))  
#     earth_radius = 6371.393  
#     s=s*earth_radius  
#     if(s<0):
#         return -s*1000
#     else:
#         return s*1000

# record should order by longitude first
def core_field_extra(stops,records):
    start = datetime.now()
    
    df =pd.DataFrame(columns = ['latitude','longitude','routeNumber','updated','plannedArrivalTime','device','segment','ATE'])
    latitude = '' 
    longitude = ''
    routeNumber = route_stops.iloc[0].routeNumber 
    updated = ''
    plannedArrivalTime = ''
    device = ''
    segment = ''
    ate = ''
    
    bus_tracker = 0 #// for bus stop change use
    for raw in records.itertuples():
        if(bus_tracker < len(stops)):
            flag = raw.latitude < stops.iloc[[bus_tracker]].latitude
            if(flag.any()):
#                     print(raw)
                    continue
            elif(bus_tracker+1 < len(stops)):
                if((raw.latitude >= stops.iloc[[bus_tracker]].latitude).any() & (raw.latitude <= stops.iloc[[bus_tracker+1]].latitude).any()):
                    latitude = raw.latitude
                    longitude = raw.longitude
                    updated = raw.updated
                    plannedArrivalTime = stops.iloc[bus_tracker+1].plannedArrivalTime
                    device = raw.device
                    segment = stops.iloc[bus_tracker,10] + '-' + stops.iloc[bus_tracker+1,10]
                    ate = 'N/A'
                    df = df.append({'latitude':latitude,'longitude':longitude,'routeNumber':routeNumber,
                                   'updated':updated,'plannedArrivalTime':plannedArrivalTime,'device':device,
                                   'segment':segment,'ATE':ate},ignore_index=True)
                else:
                    latitude = '' 
                    longitude = ''
                    updated = ''
                    plannedArrivalTime = ''
                    device = ''
                    segment = ''
                    
                    df.loc[df.ATE == 'N/A', 'ATE'] = raw.updated
                    bus_tracker= bus_tracker + 1 

#         print('sub_main_Duration:',(datetime.now() - start).total_seconds())
#     print('sub_Duration:',(datetime.now() - start).total_seconds())
    if(len(records) > 0):
        df.loc[df.ATE == 'N/A', 'ATE'] = records.iloc[-1].updated
    return df

In [21]:
# link route and stop info together
csv_route_num1 = csv_route[(csv_route.routeNumber ==1) & (csv_route.isGo2Work == True) ]
csv_route_num1 = csv_route_num1.sort_values(["seqNumber"],ascending = True)
csv_route_num1 = csv_route_num1.iloc[:,1:7]

route_stops = csv_route_num1.merge(csv_stop,left_on='stopId',right_on = '_id')

In [22]:
route_stops.head(10)

,busId,isGo2Work,plannedArrivalTime,routeNumber,seqNumber,stopId,_id,isGoToWork,latitude,longitude,stopName,updated
0,5ae8393c3023169da447c2ae,True,7:23,1,1,5ae83320748fb2a67c5f6e54,5ae83320748fb2a67c5f6e54,True,22.249541,113.520559,明珠中,2018-05-01T09:28:00.844Z
1,5ae8393c3023169da447c2ae,True,7:26,1,2,5ae8334c748fb2a67c5f6e55,5ae8334c748fb2a67c5f6e55,True,22.256653,113.519771,翠微,2018-05-01T09:28:44.366Z
2,5ae8393c3023169da447c2ae,True,7:34,1,3,5ae8336c748fb2a67c5f6e56,5ae8336c748fb2a67c5f6e56,True,22.267762,113.536134,仁恒星园,2018-05-01T09:29:16.147Z
3,5ae8393c3023169da447c2ae,True,7:37,1,4,5ae834df567e17937cf407e1,5ae834df567e17937cf407e1,True,22.268722,113.542062,香洲区府,2018-05-01T09:35:27.388Z
4,5ae8393c3023169da447c2ae,True,7:38,1,5,5ae83521567e17937cf407e2,5ae83521567e17937cf407e2,True,22.269261,113.545517,南村,2018-05-01T09:36:33.812Z
5,5ae8393c3023169da447c2ae,True,7:40,1,6,5ae8353d567e17937cf407e3,5ae8353d567e17937cf407e3,True,22.276603,113.557696,兴业中,2018-05-01T09:37:01.373Z
6,5ae8393c3023169da447c2ae,True,8:30,1,7,5af85642ac5680781c7bb90c,5af85642ac5680781c7bb90c,True,22.372631,113.575924,南方软件园,2018-05-13T15:14:10.382Z


In [23]:
#filter 2018-5-30 
csv_record_bus1 = csv_record[(csv_record['bus']== 1.0)
                              & (csv_record['device']!= 'arduino')
                              & (pd.to_datetime(csv_record['updated'],utc = True) > pd.to_datetime('2018/06/13 23:00',utc = True))
                              & (pd.to_datetime(csv_record['updated'],utc = True) < pd.to_datetime('2018/06/14 00:40',utc = True))] # filter all bus1 data

NameError: name 'csv_record' is not defined

In [ ]:
csv_record_bus1 = csv_record_bus1.sort_values(by = 'updated',ascending = True)
bus1_copy = csv_record_bus1.copy()
location = bus1_copy

location.head()

In [ ]:
plt.plot(location['longitude'],location['latitude'],'ro')

In [ ]:
plt.plot(route_stops.longitude,route_stops.latitude, "go",location['longitude'],location['latitude'],'r--')

## 2.data extra

### @functions

In [ ]:
# start_date: date time start, string format: yyyy/MM/dd HH:mm, kindly remind start HH:mm should be '23:00'
# stop_date: date time ending, string format: yyyy/MM/dd HH:mm, kindly remind stop HH:mm should be '00:40'
# records

def extra_train_test_data(start_date,stop_date,records,stops):
    # caluculate the time span of all data by day, and arrange the data  
    start = datetime.now()
    time_span = pd.to_datetime(stop_date,utc = True) - pd.to_datetime(start_date,utc = True)
    time_span = time_span.days +1


    final_df = pd.DataFrame(columns = ['latitude','longitude','routeNumber','updated','plannedArrivalTime','device','segment','ATE'])

    for i in range(time_span):
        current_start_date = pd.to_datetime(start_date,utc = True) + timedelta(days = i)
        current_stop_date = current_start_date + timedelta(hours = 1, minutes = 40)
        
#         print('current_start_date: ' + str(current_start_date))
#         print('current_stop_date' + str(current_stop_date))
        current_record_busi_all = records[(pd.to_datetime(records['updated'],utc = True) > pd.to_datetime(current_start_date,utc = True))
                              & (pd.to_datetime(records['updated'],utc = True) < pd.to_datetime(current_stop_date,utc = True))] 
        current_location = current_record_busi_all
#         print('total location: ' + str(current_location.shape))
#         print('main_sub_Duration:',(datetime.now() - start).total_seconds())
        current_extra = core_field_extra(stops,current_location)
        final_df = final_df.append(current_extra, ignore_index=True)
    
        
     # save the result to csv
    final_df.to_csv('./../data/pure_data/pure_data1.csv',sep = '|',encoding = 'utf-8') 

    print('----------------------------------')
    print('all data start date: ' + str(pd.to_datetime(csv_record_bus1_all.iloc[0].updated,utc = True)))
    print('all data ending date: ' + str(pd.to_datetime(csv_record_bus1_all.iloc[-1].updated,utc = True)))
    print('time span(/day): ' + str(time_span))
    print('Duration:',(datetime.now() - start).total_seconds())
    print('----------------------------------')
    
    return final_df

### A.sample for test

In [ ]:
#show 5.30 data extra result
_5_30 = core_field_extra(route_stops,location)
sh = _5_30[_5_30.ATE == 'N/A']
_5_30.tail()

In [ ]:
# train_data_set.to_csv('./../data/pure_data/_5_30.csv',sep = '|',encoding = 'utf-8')  # save the result to csv

### B.extra all history records

In [ ]:
# filter all stm32 data from 5,22 to 5,30
csv_record_bus1_all = csv_record[(csv_record['bus']== 1.0)
                              & (csv_record['device']!= 'arduino')
                              & (pd.to_datetime(csv_record['updated'],utc = True) > pd.to_datetime('2018/05/21 23:00',utc = True))
                              & (pd.to_datetime(csv_record['updated'],utc = True) < pd.to_datetime('2018/06/26 00:40',utc = True))] # filter all bus1 data


In [ ]:
csv_record_bus1_all = csv_record_bus1_all.sort_values(by = 'updated',ascending = True)
csv_record_bus1_all.shape

In [ ]:
# # caluculate the time span of all data by day, and arrange the data  
# time_span = pd.to_datetime(csv_record_bus1_all.iloc[-1].updated,utc = True) - pd.to_datetime(csv_record_bus1_all.iloc[0].updated,utc = True)
# time_span = time_span.days +1


# final_df = pd.DataFrame(columns = ['segment','distance(unit: m)','frequency','span(unit: s)','routeNumber'])

# for i in range(time_span):
#     current_start_date = pd.to_datetime('2018/05/21 23:00',utc = True) + timedelta(days = i)
#     current_stop_date = pd.to_datetime('2018/05/22 00:40',utc = True) + timedelta(days = i + 1)

#     current_record_busi_all = csv_record_bus1_all[(pd.to_datetime(csv_record_bus1_all['updated'],utc = True) > pd.to_datetime(current_start_date,utc = True))
#                               & (pd.to_datetime(csv_record_bus1_all['updated'],utc = True) < pd.to_datetime(current_stop_date,utc = True))] 
#     current_location = current_record_busi_all.iloc[:,2:6]
#     current_extra = core_field_extra(route_stops,current_location)
#     final_df = final_df.append(current_extra, ignore_index=True)
    
    
#  # save the result to csv
# final_df.to_csv('./../data/pure_data/pure_data.csv',sep = '|',encoding = 'utf-8') 

# print('----------------------------------')
# print('all data start date: ' + str(pd.to_datetime(csv_record_bus1_all.iloc[0].updated,utc = True)))
# print('all data ending date: ' + str(pd.to_datetime(csv_record_bus1_all.iloc[-1].updated,utc = True)))
# print('time span(/day): ' + str(time_span))
# print('----------------------------------')


In [ ]:
# final_df = final_df.sort_values(by = 'segment',ascending = True)

In [ ]:
final_df = extra_train_test_data('2018/05/21 23:00','2018/06/26 00:40',csv_record_bus1_all,route_stops)
print(final_df.shape)
final_df.head()

### C.split the whole analysis data into train set and test set

In [ ]:
# use 2018/05/21 - 2018/06/18 data as training data
# use 2018/06/18 - 2018/06/26 data as testing data
train_data_set = extra_train_test_data('2018/05/21 23:00','2018/06/18 00:40',csv_record_bus1_all,route_stops)
test_data_set = extra_train_test_data('2018/06/18 23:00','2018/06/26 00:40',csv_record_bus1_all,route_stops)

print('train_data_set shape: ' + str(train_data_set.shape))
print('test_data_set shape: ' + str(test_data_set.shape))

## 3.Machine Learning

### @functions

In [ ]:
train_data_set.replace({'segment': { '仁恒星园-香洲区府': 0,'兴业中-南方软件园': 1,'南村-兴业中': 2,
                                    '明珠中-翠微': 3,'翠微-仁恒星园': 4,'香洲区府-南村': 5},
                        'device':{'stm32': 0,'hcb': 1}}, inplace=True)


In [ ]:
test_data_set.replace({'segment': { '仁恒星园-香洲区府': 0,'兴业中-南方软件园': 1,'南村-兴业中': 2,
                                    '明珠中-翠微': 3,'翠微-仁恒星园': 4,'香洲区府-南村': 5},
                        'device':{'stm32': 0,'hcb': 1}}, inplace=True)


In [ ]:
train_data_set.updated = pd.to_datetime(train_data_set.updated,utc = True)
test_data_set.updated = pd.to_datetime(test_data_set.updated,utc = True)

In [ ]:
train_data_set.ATE = pd.to_datetime(train_data_set.ATE,utc = True)
test_data_set.ATE = pd.to_datetime(test_data_set.ATE,utc = True)

In [ ]:
train_data_set['ATE_span'] = train_data_set.apply(lambda x: (x.ATE - x.updated).total_seconds(), axis=1)

In [ ]:
test_data_set['ATE_span'] = test_data_set.apply(lambda x: (x.ATE - x.updated).total_seconds(), axis=1)

In [ ]:
train_data_set.head()

In [ ]:
test_data_set.head()

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LogisticRegression,LinearRegression,Ridge
from sklearn.metrics import mean_squared_error
from sklearn.feature_extraction import DictVectorizer

# ['latitude','longitude','routeNumber','device','segment']
label_list = ['latitude','longitude','segment']

X_train=train_data_set[label_list]
Y_train=train_data_set.ATE_span
X_test=test_data_set[label_list]
Y_test=test_data_set.ATE_span
random_forest=RandomForestRegressor()
random_forest.fit(X_train, Y_train)
preds_test = random_forest.predict(X_test)

train_score = mean_squared_error(Y_test, preds_test) ** 0.5
print(train_score / 60)